<a href="https://colab.research.google.com/github/Isaac-Jacobson/snowFlakeDetector/blob/main/SnowflakeClassifier_Binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Abonded this notebook
I figured just using a object detector would be easier and better than have a detector (possibly hard coded) and a classifier.

#Binary classifier
For testing if an object is a snowflake or not

In [ ]:
#install -q tflite-model-maker
#!pip install -q tflite-support

In [ ]:
import tensorflow as tf
import matplotlib as plt
import panda as pd
import numpy as np
import os
import tarfile
import urllib

from tflite_model_maker import object_detector


In [ ]:
trainData, validData, testData = object_detector.DataLoader.from_csv('./annotations.csv')

# Using Stochastic Gradient Descent
Mostly for evaluating precision vs recall

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(trainData, trainLabel)

In [ ]:
from sklearn.model_selection import cross_val_predict
from scklearn.metrics import confusion_matrix, precision_score, recall_score

pred = cross_val_predict(sgd_clf, testData, testLabels)
confusion_matrix(testLabels, pred)
precision_score(testLabels, pred)
recall_score(testLabels, pred)

# Using A Neural Network (Autokeras)
Here I used autokeras makming it a little more accurate but more time consuming

In [ ]:
#!pip install autokeras
import autokeras as ak

In [ ]:
# Initialize the image classifier.
input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    block_type="resnet", # Search resnet architectures (for now)
    normalize=True,
    augment=False,
)(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=12
)
# Feed the image classifier with training data.
clf.fit(trainData, trainLabels) # unspecified number of epochs

# Predict with the best model.
pred = clf.predict(testData)
print(pred)

# Evaluate the best model with testing data.
print(clf.evaluate(testData, testLabels))